In [ ]:
import soccerdata as sd
import matplotlib.pyplot as plt
import sklearn as skl
import pandas as pd
import numpy as np
import pyarrow
import seaborn as sb
import csv
import os
from mplsoccer import VerticalPitch, Pitch
import math

In [ ]:
DIR = 'utd_games'
num = sum(os.path.isfile(os.path.join(DIR, name)) for name in os.listdir(DIR))
num

In [ ]:
os.listdir(DIR)[0]

In [ ]:
games = []
for i in range(num):
    games.append(os.listdir(DIR)[i])

In [ ]:
game = pd.read_csv(DIR+'/'+games[0])
game



In [ ]:
def shotmap_gen(match):
    DIR = 'utd_games'
    game = pd.read_csv(DIR+'/'+match)

    shots_df = game[['minute','second','team','x', 'y' ,'is_shot', 'is_goal', 'blocked_x', 'blocked_y','goal_mouth_y','goal_mouth_z']]
    shots_df = shots_df[shots_df['is_shot'] == True]
    shots_df

    teams = shots_df['team'].sort_values().to_list()
    opposition = ''
    if teams[0] == 'Man Utd':
        opposition = teams[-1]
    else:
        opposition = teams[0]

    man_utd_shots_df = shots_df[shots_df['team'] == 'Man Utd']

    opposition_shots_df = shots_df[shots_df['team'] == opposition]

    man_utd_shots_df

    mutd_g_df = man_utd_shots_df[man_utd_shots_df['is_goal'] == True]
    mutd_ng_df = man_utd_shots_df[man_utd_shots_df['is_goal'] != True]

    opposition_g_df = opposition_shots_df[opposition_shots_df['is_goal'] == True]
    opposition_ng_df = opposition_shots_df[opposition_shots_df['is_goal'] != True]

    pitch = Pitch(half=False, goal_type='box', goal_alpha=0.8)
    fig, ax = pitch.draw(figsize=(12, 10))
    #whoscored coordinates are 0 ~ 100. defauly pitch on Pitch is 0 ~ 120, so conversion by *1.2 and *0.8 is required
    United_sc_g = pitch.scatter(mutd_g_df['x']*1.2, mutd_g_df['y']*0.8, marker="o",c= 'red', ax=ax, linewidth = 3)
    United_sc_g = pitch.scatter(mutd_ng_df['x']*1.2, mutd_ng_df['y']*0.8, marker="o",c= 'white', ax=ax, linewidth = 3)
    Opposition_sc_g = pitch.scatter(120 - opposition_g_df['x']*1.2, 80 - opposition_g_df['y']*0.8, marker="o",c= 'blue', ax=ax, linewidth = 5)
    Opposition_sc_g = pitch.scatter(120 - opposition_ng_df['x']*1.2, 80 - opposition_ng_df['y']*0.8, marker="o",c= 'green', ax=ax, linewidth = 5)

    fig.text(s=f"Manchester United vs {opposition} Shots Map", x= 0.06, y= 0.96, fontsize=22,fontweight="bold")
    ax.legend(title = 'Outcome', labelspacing=1, loc="upper center",ncol=6,frameon=True,fancybox=True, shadow=True,bbox_to_anchor=(0.5, -0.01))

In [ ]:
for i in range(len(games)):
    shotmap_gen(games[i])
    i+=1